# Content Moderation

This notebook contains example snippets for moderating image files.

Confirm that the `boto3` module is installed.

In [5]:
!pip3 install boto3

You should consider upgrading via the 'c:\tools\python3.8\python.exe -m pip install --upgrade pip' command.


Configure the global variables for this notebook.

1. Set the `bucket_name` to an Amazon S3 bucket within your AWS account
1. Set the `region_name` to the AWS Region containing the bucket
1. Create the `rekognition` client for the same region

In [6]:
import boto3

bucket_name = 'cv-on-aws-book-nbachmei'
region_name = 'us-east-2'
rekognition = boto3.client('rekognition', region_name=region_name)

The `moderate_image` function passes an Amazon S3 location to the `DetectModerationLabels` API.

In [ ]:
photo='06_ContentModeration/images/swimwear.jpg'

def moderate_image(photo, bucket):    
    response = rekognition.detect_moderation_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}})

    print('Detected labels for ' + photo)    
    for label in response['ModerationLabels']:
        print (label['Name'] + ' : ' + str(label['Confidence']))
        print (label['ParentName'])
    return len(response['ModerationLabels'])

def main():    
    label_count=moderate_image(photo, bucket_name)
    print("Labels detected: " + str(label_count))

You can define policies that use the `detect_moderation_reponse` to confirm concepts are present or missing. For example, the `contains_alcohol` function returns **True** for any `top_level` category of Alcohol

In [ ]:
def contains_alcohol(detect_moderation_response):
    for label in detect_moderation_response['ModerationLabels']:
        top_level = label['ParentName']
        confidence = label['Confidence']

        if top_level == "Alcohol":
            print('Alcohol detected - %2.2f%%.' % confidence)
            return False

    return True

You can also define more fine-grained policies by targeting the label's `secondary_level`.

In [ ]:
ALLOWED_SUGGESTIVE_LABELS= [
    'Female Swimwear Or Underwear',
    'Male Swimwear Or Underwear']

def contains_appropriate_attire(detect_moderation_response):
    for label in detect_moderation_response['ModerationLabels']:
        top_level = label['ParentName']
        secondary_level = label['Name']
        confidence = label['Confidence']

        if top_level == "Explicit Nudity":
            print('Explicit Nudity detected - %2.2f%%.' % confidence)
            return False

        if top_level == "Suggestive":
            if secondary_level not in ALLOWED_SUGGESTIVE_LABELS:
                print('Prohibited Suggestive[%s] detected - %2.2f%%.' % 
                    (secondary_level, confidence))
                return False
            else:                
                print('Allowed Suggestive[%s] detected - %2.2f%%.' % 
                    (secondary_level, confidence))

    return True

Let's combine these polices into a simple check. Notice how this pattern results in very readable definitions!

```
if not contains_alcohol(response) and contains_appropriate_attire(response):
        return True
    return False
```

In [ ]:
import boto3

bucket_name = 'cv-on-aws-book-nbachmei'
photo='06_ContentModeration/images/swimwear.jpg'
region_name = 'us-east-2'
rekognition = boto3.client('rekognition', region_name=region_name)

ALLOWED_SUGGESTIVE_LABELS= [
    'Female Swimwear Or Underwear',
    'Male Swimwear Or Underwear']

def contains_appropriate_attire(detect_moderation_response):
    for label in detect_moderation_response['ModerationLabels']:
        top_level = label['ParentName']
        secondary_level = label['Name']
        confidence = label['Confidence']

        if top_level == "Explicit Nudity":
            print('Explicit Nudity detected - %2.2f%%.' % confidence)
            return False

        if top_level == "Suggestive":
            if secondary_level not in ALLOWED_SUGGESTIVE_LABELS:
                print('Prohibited Suggestive[%s] detected - %2.2f%%.' % 
                    (secondary_level, confidence))
                return False
            else:                
                print('Allowed Suggestive[%s] detected - %2.2f%%.' % 
                    (secondary_level, confidence))

    return True

def contains_alcohol(detect_moderation_response):
    for label in detect_moderation_response['ModerationLabels']:
        top_level = label['ParentName']
        confidence = label['Confidence']

        if top_level == "Alcohol":
            print('Alcohol detected - %2.2f%%.' % confidence)
            return False

    return True  

def moderate_image(photo, bucket):    
    response = rekognition.detect_moderation_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}})

    if not contains_alcohol(response) and contains_appropriate_attire(response):
        return True
    return False

def main():    
    label_count=moderate_image(photo, bucket_name)
    print("Labels detected: " + str(label_count))